In [13]:
from construct import *
import enum

data = None
tiff_file = 'data/data.NEF'
with open(tiff_file, 'rb') as f:
    data = f.read()

# TIFF (Tag Image File Format)

参考标准 [TIFF6.0](https://web.archive.org/web/20180810205359/https://www.adobe.io/content/udp/en/open/standards/TIFF/_jcr_content/contentbody/download/file.res/TIFF6.pdf)

# TIFF Struct

TIFF 文件最大为 2**32 bytes, 也就是 4GB.

TIFF 由一个 **Image File Header** 和若干个 **Image File Directory(IFD)** 以及实际的数据组成

结构图如下:

![图 1](../asset/bf5757e490b74e3036f7972526d2a99880ec61076cf0aa2a13c01d45854089ac.png)  

## Image File Header

头部信息一共 8 个字节

- Bytes 0-1: 指示大小端
    - II(0x49 0x49): 小端字节序
    - MM(0x4D 0x4D): 大端字节序
- Bytes 2-3: 固定值 42, 进一步标识该文件为 TIFF 文件
    - 小端序: 0x2A 0x00
    - 大端序: 0x00 0x2A
- Bytes 4-7: 指向第一个 IFD 的 offset, 第一个 IFD 可能存在于 Image File Header 之后的任意地方, 但其偏移必须是偶数

![图 2](../asset/4371d66ff58c949220a99dd41aa537f9404921f1914bd995d0ecb6f6bab07a43.png)  


## Image File Directory(IFD)

每个 TIFF 文件必须要有至少一个 IFD, 而每个 IFD 至少有一个 IFD Entry 

IFD的结构为:

- Bytes 0-1: 指示 IFD 中包含 IFD Entry 的数量, 这里假设为 B
- 随后的 12 * B 个字节: 这里包含了 B 个 DE, 每个 DE 大小为 12 Bytes
- 随后的 4 个字节: 下一个 IFD 的偏移量, 如果为 0, 则表示当前为最后一个 IFD 

![图 1](../asset/bf5757e490b74e3036f7972526d2a99880ec61076cf0aa2a13c01d45854089ac.png)  

### IFD Entry

每个 IFD Entry 为 12 Bytes

结构如下:
- Bytes 0-1: Tag
- Bytes 2-3: Type
- Bytes 4-7: Value 的数量
- Bytes 8-11: Value 的值 或 Value 的起始 offset, 取决于 Type, offset 可以在文件的任何地方, offset 必须是偶数

![图 3](../asset/2e76318ec5802dd5faeebad564cce5fd2b35b16e95b76ae3b2cb3f7c8b3029ce.png)  

#### Type

| Type |   description |
| --- | --- |
| 1=BYTE | 8-bit unsigned integer. |
| 2=ASCII | 8-bit byte that contains a 7-bit ASCII code; the last byte shall be NUL (binary zero). |
| 3=SHORT | 16-bit (2-byte) unsigned integer. |
| 4=LONG | 32-bit (4-byte) unsigned integer. |
| 5=RATIONAL | Two LONGs: the first represents the numerator of a fraction; the second, the denominator. |
| 6=SBYTE | An 8-bit signed (twos-complement) integer. |
| 7=UNDEFINED | An 8-bit byte that may contain anything, depending on the definition of the field. |
| 8=SSHORT | A 16-bit (2-byte) signed (twos-complement) integer. |
| 9=SLONG | A 32-bit (4-byte) signed (twos-complement) integer. |
| 10=SRATIONAL | Two SLONGs: the first represents the numerator of the fraction, the second the denominator. |
| 11=FLOAT | Single precision (4-byte) IEEE format. |
| 12=DOUBLE | Double precision (8-byte) IEEE format. |

In [64]:
%run TiffTag.py

class TagType(enum.IntEnum):
    BYTE = 1
    ASCII = 2
    SHORT = 3
    LONG = 4
    RATIONAL = 5
    SBYTE = 6
    UNDEFINED = 7
    SSHORT = 8
    SLONG = 9
    SRATIONAL = 10
    FLOAT = 11
    DOUBLE = 12

kTagFmt = {
    TagType.BYTE : Byte,
    TagType.ASCII : Byte,
    TagType.SHORT : Short,
    # TODO: finish this
}

class Tag:
    def __init__(self, entry, data, is_little):
        self.data = data
        self.is_little = is_little
        self.tag = entry.Tag
        self.type = TagType(entry.Type)
        self.count = entry.Count
        self.name = TAG[self.tag] if self.tag in TAG else 'Unknown'
        self.value = self.parse_value(entry.Data)

    def parse_value(self, raw_data):
        try:
            if self.count == 1:
                return [raw_data]
            elif self.type == TagType.LONG:
                return list(Array(self.count, BytesInteger(4, swapped=self.is_little)).parse(self.data[raw_data:]))
            elif self.type == TagType.SHORT:
                return list(Array(self.count, BytesInteger(2, swapped=self.is_little)).parse(self.data[raw_data:]))
            else:
                return [raw_data]
        except:
            return [raw_data]
        

    def __str__(self):
        ret = f'{self.name}({self.tag}/{hex(self.tag)}) {self.type.name} count:{self.count} {self.value}\n'
        return ret

class IFD:
    def __init__(self, data, offset, is_little):
        self.data = data
        self.is_little = is_little
        
        self.IFD_entry_fmt = Struct(
            "Tag" / BytesInteger(2, swapped=self.is_little),
            "Type" / BytesInteger(2, swapped=self.is_little),
            "Count" / BytesInteger(4, swapped=self.is_little),
            "Data" / BytesInteger(4, swapped=self.is_little),
        )

        self.IFD_fmt = Struct(
            "IFDEntryCount" / BytesInteger(2, swapped=self.is_little),
            "IFDEntries" / Array(this.IFDEntryCount, self.IFD_entry_fmt),
            "NextIFDOffset" / BytesInteger(4, swapped=self.is_little)
        )

        self.ifd = self.IFD_fmt.parse(self.data[offset:])
        self.tags = self.parse_tags()

        self.sub_ifds = self.get_sub_ifds()

    def get_sub_ifds(self):
        sub_offset_tag = self.get_tag(330)
        if sub_offset_tag == None: 
            return None
        sub_ifd_offsets = sub_offset_tag.value
        return [IFD(self.data, offset, self.is_little) for offset in sub_ifd_offsets]

    def get_tag(self, tag):
        try:
            return self.tags[tag]
        except KeyError:
            return None

    def next(self):
        next_offset = self.ifd.NextIFDOffset
        if next_offset != 0:
            return IFD(self.data, next_offset, self.is_little)
        else:
            return None

    def parse_tags(self):
        tags_map = {}
        for entry in self.ifd.IFDEntries:
            tags_map[entry.Tag] = Tag(entry, self.data, self.is_little)
        return tags_map
    
    def __str__(self):
        ret = ''
        ret += f'tag count: {len(self.tags)}\n'
        for tag in self.tags.values():
            ret += f'{tag}\n'
        return ret 


class TIFF:
    def __init__(self, data):
        self.data = data
        self.is_little = data[:2] == b'II'

        self.header_fmt = Struct(
            "ByteOrder" / Bytes(2),
            "MagicNum" / BytesInteger(2, swapped=self.is_little),
            "FirstIFDOffset" / BytesInteger(4, swapped=self.is_little),
        )

        self.header = self.header_fmt.parse(self.data)
        offset = self.header.FirstIFDOffset
        self.ifds = self.parse_ifds(offset)

    def parse_ifds(self, offset, data=None):
        if data == None:
            data = self.data
        ifd = IFD(data, offset, self.is_little)
        ifds = [ifd]
        while True:
            ifd = ifd.next()

            if ifd == None:
                break
            else:
                ifds.append(ifd)

        return ifds

    def __str__(self):
        ret = f'Header {self.header.__str__()}\n'
        ret += f'IFDs list len: {len(self.ifds)}\n'
        ret += ('-'*50 + '\n')
        for idx, ifd in enumerate(self.ifds):
            ret += f'--- IFD#{idx} ---\n'
            ret += f'{ifd}\n'

        return ret

tiff = TIFF(data)
print(tiff)
# print(tiff.sub_IFDs)

Header Container: 
    ByteOrder = b'II' (total 2)
    MagicNum = 42
    FirstIFDOffset = 8
IFDs list len: 1
--------------------------------------------------
--- IFD#0 ---
tag count: 25
NewSubfileType(254/0xfe) LONG count:1 [1]

ImageWidth(256/0x100) LONG count:1 [160]

ImageLength(257/0x101) LONG count:1 [120]

BitsPerSample(258/0x102) SHORT count:3 [8, 8, 8]

Compression(259/0x103) SHORT count:1 [1]

PhotometricInterpretation(262/0x106) SHORT count:1 [2]

Make(271/0x10f) ASCII count:18 [324]

Model(272/0x110) ASCII count:11 [344]

StripOffsets(273/0x111) LONG count:1 [946870]

Orientation(274/0x112) SHORT count:1 [1]

SamplesPerPixel(277/0x115) SHORT count:1 [3]

RowsPerStrip(278/0x116) LONG count:1 [120]

StripByteCounts(279/0x117) LONG count:1 [57600]

XResolution(282/0x11a) RATIONAL count:1 [356]

YResolution(283/0x11b) RATIONAL count:1 [364]

PlanarConfiguration(284/0x11c) SHORT count:1 [1]

ResolutionUnit(296/0x128) SHORT count:1 [2]

Software(305/0x131) ASCII count:11 [372]



# TIFF/EP (Tag Image File Format / Electronic Photography)

TIFF/EP 是基于 TIFF6 标准为相机图片扩展的一个标准. 
为了完全兼容 TIFF 6.0 的解析, 其一般会在第一个 IFD 中包含一个未压缩的缩略图, 
该缩略图一般大小为 120 * 160, 存储于一个 strip 中.

TIFF/EP 的标识符为 
- Tag 37398 (TIFF/EPstandardID)

其主要数据都存储在第一个 IFD 中的 SubIFDs 中
- Tag 330 (SubIFDs)

同时其规定, TIFF/EP 中每一个图片数据段(tile 或 strip)都包含一个完整的 JPEG 数据流

In [40]:
class TIFF_EP(TIFF):
    def __init__(self, data):
        super().__init__(data)

        self.exifs = [IFD(self.data, offset, self.is_little) for offset in self.ifds[0].get_tag(34665).value]

tiff_ep = TIFF_EP(data)
print(tiff_ep)


tag count: 33
ExposureTime(33434/0x829a) RATIONAL count:1 [2932]

FNumber(33437/0x829d) RATIONAL count:1 [2940]

ExposureProgram(34850/0x8822) SHORT count:1 [0]

PhotographicSensitivity(34855/0x8827) SHORT count:1 [800]

SensitivityType(34864/0x8830) SHORT count:1 [2]

TVX_ExposureTime(36867/0x9003) ASCII count:20 [2948]

TVX_BackgroundTime(36868/0x9004) ASCII count:20 [2968]

ExposureBiasValue(37380/0x9204) SRATIONAL count:1 [2988]

MaxApertureValue(37381/0x9205) RATIONAL count:1 [2996]

MeteringMode(37383/0x9207) SHORT count:1 [5]

LightSource(37384/0x9208) SHORT count:1 [0]

Flash(37385/0x9209) SHORT count:1 [0]

FocalLength(37386/0x920a) RATIONAL count:1 [3004]

MakerNote(37500/0x927c) UNDEFINED count:943778 [3092]

SubsecTime(37520/0x9290) ASCII count:3 [12336]

SubsecTimeOriginal(37521/0x9291) ASCII count:3 [12336]

SubsecTimeDigitized(37522/0x9292) ASCII count:3 [12336]

SensingMethod(41495/0xa217) SHORT count:1 [2]

FileSource(41728/0xa300) UNDEFINED count:1 [3]

SceneType(4172

## Nikon Electronic File (NEF)

NEF 格式时 Nikon 基于 TIFF/EP 标准扩展的自家的相机图片格式.
参考资料: [nef](http://lclevy.free.fr/nef/)

其主要的改动在于, SubIFDs(Tag 330) 中, 有两个偏移量
- 第一个 SubIFD 存储了一张全尺寸有损压缩的 JPEG 格式图片
- 第二个 SubIFD 存储了一张全尺寸无损压缩的 RAW 格式图片

### 0th IFD

其第一个 IFD 的定义如下:

| Tag value	| Name	| Type	| Length	| Description |
| --------- | ----- | ----- | --------- | ----------- |
| 0x00fe/254	| SubfileType	| 4	| 1	| 1=Reduced-resolution image |
| 0x0100/256	| imageWidth	| 4	| 1	| 160 |
| 0x0101/257	| ImageHeight	| 4	| 1	| 120 |
| 0x0102/258	| BitsPerSample	| 3	| 3	| [ 8, 8, 8 ] |
| 0x0103/259	| Compression	| 3	| 1	| 1=uncompressed |
| ...	| ...	| 	| 	| ... |
| 0x014a / 330	| SubIFD tag	| 4	| 2	| [ JpegImageOffset, RawOffset ] : offsets to the 2 child IFDs |
| 0x0214 / 34665	| ReferenceBlackWhite	| 5=rational	| 6	|  |
| 0x8769 / 34665	| EXIF	| 4	| 1	| offset to the EXIF IFD. the EXIF IFD contains a pointer to the Makernote IFD |
| 0x9286 / 37510	| UserComment	| 7	| variable	|  |
| ...	| ...	| 	| 	| ... |

### 0th subIFD (存储有损JPEG图片)

| Tag value	| Name	| Type	| Length	| Description |
| --------- | ----- | ----- | --------- | ----------- |
| 0x00fe / 254	| SubfileType	| 4	| 1	| 1=Reduced-resolution image |
| 0x0103 / 259	| Compression	| 3	| 1	| 6=old/jpeg |
| 0x011a / 282	| XResolution	| 5=rational	| 1	| 300 |
| 0x011b / 283	| YResolution	| 5	| 1	| 300 |
| 0x0128 / 296	| ResolutionUnit	| 3	| 1	| 2=pixel_per_inch |
| 0x0201 / 513	| JpgFromRawStart	| 4	| 1	| offset to image data |
| 0x0202 / 514	| JpgFromRawLength	| 4	| 1	| image data length |
| 0x0213 / 531	| YCbCrPositioning	| 3	| 1	| 2=co_sited |

### 1th subIFD (存储无损RAW图片)

| Tag value	| Name	| Type	| Length	| Description |
| --------- | ----- | ----- | --------- | ----------- |
| 0x00fe / 254	| SubfileType	| 4	| 1	| 0=Full-resolution Image |
| 0x0100 / 256	| ImageWidth	| 4	| 1	| 3904 for the D60 |
| 0x0101 / 257	| ImageHeight	| 4	| 1	| 2616 for the D60 |
| 0x0102 / 258	| BitsPerSample	| 1	| 1	| 12bits for the D60 |
| 0x0103 / 259	| Compression	| 3	| 1	| 1=uncompressed, 34713=Nikon NEF Compressed [[Details](http://lclevy.free.fr/nef/nikon_compression.c), [Values](http://lclevy.free.fr/nef/values.txt)] |
| 0x0106 / 262	| PhotometricInterpretation	| 3	| 1	| 32803=Color Filter Array |
| 0x0111 / 273	| JpgFromRawStart	| 4	| 1	| offset to the image data |
| 0x0115 / 277	| SamplesPerPixel	| 3	| 1	| 1 |
| 0x0116 / 278	| RowsPerStrip	| 3	| 1	| 2616 for the D60 |
| 0x0117 / 279	| JpgFromRawLength	| 4	| 1	| image data lenght |
| 0x011a / 282	| XResolution	| 5=rational	| 1	| 300 |
| 0x011b / 283	| YResolution	| 5	| 1	| 300 |
| 0x011c / 284	| PlanarConfiguration	| 3	| 1	| 1 = Chunky |
| 0x0128 / 296	| ResolutionUnit	| 3	| 1	| 2=pixel_per_inch |
| 0x828d / 33421	| CFARepeatPatternDim	| 3	| 2	| [2, 2] = 2x2 |
| 0x828e / 33422	| CFAPattern2	| 1	| 4	| [1, 2, 0, 1] = [G, B, R, G] for the D60 |
| 0x9217 / 37399	| SensingMethod	| 3	| 1	| 2 = One-chip color area (D60) |

### Makernote

Makernote 属于是 Nikon 自定义的字段了, 里面包含了一个 jpeg 的缩略图, 
同时Makernote本身包含了 tiff 6.0 的完整格式

其位于 NEF 的 EXIF IFD(0x8769 / 34665) 的 MakerNote tag(0x927C / 37500) 中,
MakerNote tag 的 value 为 Makernote 的 offset, 其 Count 为 Makernote 的字节数

Nikon 的缩略图就存放在 Makernote 的 0th ifd 的 NikonPreview(0x0011 / 17) 中

#### Makernote header

| Offset	| Length	| Type	| Description	| Value |
| --------- | --------- | ----- | ------------- | ----- |
| 0x0000	| 6	| string	| magic value	| "Nikon", zero terminated |
| 0x0006	| 1	| short	| version ?	| 0x0210 |
| 0x0008	| 1	| short	| ?	| 0x0000 |
| 0x000a	| 1	| short	| byte order	| Usually 0x4D4D / "MM", except for E5700 (0x4949 / "II") |
| 0x000c	| 1	| short	| TIFF magic value	| 0x2a |
| 0x00e	| 1	| long	| TIFF offset	| 8 |
| 0x0012	| 	| IFD	| first IFD	|  |

#### Makernote tags

这里的数据存储与 first IFD 中, Nikon 的所有自定义 tag 参考 [Nikon tags](https://exiftool.org/TagNames/Nikon.html)


| Tag value	| Name	| Type	| Length	| Description |
| --------- | --------- | ----- | ------------- | ----- |
| 0x0001 / 1	| MakerNoteVersion	| 7	| 4	| version="0210" for the D60 |
| 0x0002 / 2	| ISO	| 3	| 2	|  |
| 0x000e / 14	| ExposureDifference	| 7	| 4	|  |
| 0x0011 / 17	| NikonPreview	| 4	| 1	| offset to this IFD |
| 0x0012 / 18	| FlashExposureComp	| 7	| 4	|  |
| 0x001d / 27	| SerialNumber	| 3	| 7	| Serial numner (used for Color Balance decryption) |
| 0x0093 / 147	| NEF Compression	| 3	| 	| 1=lossy type1  3=lossless 4=lossy type 2 (d90, d3, d700, d300 ) |
| 0x0096 / 150	| Linearization table	| bytes	| 	| see format below |
| 0x0097 / 151	| Color Balance	| 7	| 572 for D60	| Color Balance (see decryption code below for versionn >= 200) |
| 0x00a7 / 167	| ShutterCount	| 4	| 1	| shutter count. used to decrypt While Balance tag |
| 0x00bb / 187	| ?	| 7	| 6	|  |


#### Makernote Nikon preview tag (0x0011)

Nikon 的缩略图就存在这里

| Tag value	| Name	| Type	| Length	| Description |
| --------- | --------- | ----- | ------------- | ----- |
| 0x0103 / 259	| Compression	| 3	| 1	| 6=old/jpeg |
| 0x011a / 282	| XResolution	| 5=rational	| 1	| 300 |
| 0x011b / 283	| YResolution	| 5	| 1	| 300 |
| 0x0128 / 296	| ResolutionUnit	| 3	| 1	| 2=pixel_per_inch |
| 0x0201 / 513	| JpgFromRawStart	| 4	| 1	| offset to image data |
| 0x0202 / 514	| JpgFromRawLength	| 4	| 1	| image data length |
| 0x0213 / 531	| YCbCrPositioning	| 3	| 1	| 2=co_sited |

In [60]:
class Makernote_Nikon(TIFF):
    def __init__(self, data):
        super().__init__(data)

    def get_jpeg_image(self):
        preview_ifds_offset = self.ifds[0].get_tag(17).value[0]
        preview_ifds = self.parse_ifds(preview_ifds_offset)
        jpeg_offset = preview_ifds[0].get_tag(513).value[0]
        jpeg_length = preview_ifds[0].get_tag(514).value[0]

        return self.data[jpeg_offset:jpeg_length]


class NEF(TIFF_EP):
    def __init__(self, data):
        super().__init__(data)

        self.jpeg = self.get_jpeg_image()

        self.raw_compression = self.ifds[0].sub_ifds[1].get_tag(259).value[0]

        self.makernote_fmt = Struct(
            "magic_val" / Const(b"Nikon\0"),
            "version" / Bytes(2),
            Bytes(2),
            "tiff_data" / GreedyBytes,
        )

        makernote_tag = self.exifs[0].get_tag(37500)
        makernote_offset = makernote_tag.value[0]
        makernote_len = makernote_tag.count
        makernote_data = self.data[makernote_offset : makernote_offset + makernote_len]
        self.makernote_header = self.makernote_fmt.parse(makernote_data)

        self.makernote_tiff = Makernote_Nikon(self.makernote_header.tiff_data)
        self.thumb_jpeg = self.get_jpeg_thumb()

    def get_jpeg_image(self):
        jpeg_offset = self.ifds[0].sub_ifds[0].get_tag(513).value
        jpeg_length = self.ifds[0].sub_ifds[0].get_tag(514).value

        return self.data[jpeg_offset[0] : jpeg_offset[0] + jpeg_length[0]]

    def get_jpeg_thumb(self):
        return self.makernote_tiff.get_jpeg_image()

    def extract_jpeg(self, out_path):
        with open(out_path, "wb") as f:
            f.write(self.jpeg)

    def extract_thumb_jpeg(self, out_path):
        with open(out_path, "wb") as f:
            f.write(self.thumb_jpeg)

    def __str__(self):
        ret = f"NEF raw image compression type: {self.raw_compression}\n"
        ret += f'{"+"*50}\n'
        ret += "markernote tags:\n"
        ret += f"{self.makernote_tiff}\n"
        ret += f'{"+"*50}\n'
        ret += super().__str__()
        return ret


nef = NEF(data)
nef.extract_jpeg("./data/data_nef.jpg")
nef.extract_thumb_jpeg("./data/data_nef_thumb.jpg")
print(nef)


NEF raw image compression type: 34713
++++++++++++++++++++++++++++++++++++++++++++++++++
markernote tags:
Header Container: 
    ByteOrder = b'II' (total 2)
    MagicNum = 42
    FirstIFDOffset = 8
IFDs list len: 1
--------------------------------------------------
tag count: 47
Unknown(1/0x1) UNDEFINED count:4 [808530480]

Unknown(2/0x2) SHORT count:2 [52428800]

Unknown(4/0x4) ASCII count:8 [578]

Unknown(5/0x5) ASCII count:13 [586]

Unknown(7/0x7) ASCII count:7 [602]

Unknown(8/0x8) ASCII count:13 [610]

Unknown(9/0x9) ASCII count:20 [626]

ProcessingSoftware(11/0xb) SSHORT count:2 [0]

Unknown(12/0xc) RATIONAL count:4 [646]

Unknown(13/0xd) UNDEFINED count:4 [393472]

Unknown(14/0xe) UNDEFINED count:4 [786688]

Unknown(17/0x11) LONG count:1 [20330]

Unknown(18/0x12) UNDEFINED count:4 [393472]

Unknown(27/0x1b) SHORT count:7 [678]

Unknown(29/0x1d) ASCII count:9 [694]

Unknown(30/0x1e) SHORT count:1 [1]

Unknown(31/0x1f) UNDEFINED count:8 [706]

Unknown(34/0x22) SHORT count:1 [65535

## CR2



In [88]:
cr2_data = None
# with open(r'D:\Users\Documents\WXWork\1688851908246798\Cache\File\2022-08\Canon_Canon PowerShot SX50 HS_2022-03-14_36569088.CR2', 'rb') as f:
with open(r'M:\DB\WhatsApp_Transfer\TestDatabase\Android to IOS Android WhatsApp business源数据\samsung(SM-N770F)\Attachments\sdcard\Android\media\com.whatsapp.w4b\WhatsApp Business\Media\WhatsApp Business Documents\Sent\6zwji9_lukashrtng.cr2', 'rb') as f:
    cr2_data = f.read()

cr2 = TIFF_EP(cr2_data)
print(cr2)
print(cr2.exifs[0])

# rgb_data = None
# with open(r'./data/cr2_rgb', 'rb') as f:
#     rgb_data = f.read()

# rgb_fmt = GreedyRange(Array(3, Int16ul))

# import cv2
# import numpy as np
# import matplotlib.pyplot as plt

# rgb_st = rgb_fmt.parse(rgb_data)
# rgb = np.array(list(rgb_st), np.uint16)

# rgb = np.reshape(rgb, (375, 500, 3))
# rgb = cv2.normalize(rgb, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
# print(rgb)
# print(rgb.shape)
# plt.imshow(rgb, cmap='gray')



Header Container: 
    ByteOrder = b'II' (total 2)
    MagicNum = 42
    FirstIFDOffset = 16
IFDs list len: 4
--------------------------------------------------
tag count: 18
ImageWidth(256/0x100) SHORT count:1 [5184]

ImageLength(257/0x101) SHORT count:1 [3456]

BitsPerSample(258/0x102) SHORT count:3 [8, 8, 8]

Compression(259/0x103) SHORT count:1 [6]

Make(271/0x10f) ASCII count:6 [244]

Model(272/0x110) ASCII count:13 [250]

StripOffsets(273/0x111) LONG count:1 [72176]

Orientation(274/0x112) SHORT count:1 [1]

StripByteCounts(279/0x117) LONG count:1 [2853636]

XResolution(282/0x11a) RATIONAL count:1 [282]

YResolution(283/0x11b) RATIONAL count:1 [290]

ResolutionUnit(296/0x128) SHORT count:1 [2]

DateTime(306/0x132) ASCII count:20 [298]

Artist(315/0x13b) ASCII count:18 [318]

XMP(700/0x2bc) BYTE count:8192 [49064]

Copyright(33432/0x8298) ASCII count:4 [4934209]

ExifTag(34665/0x8769) LONG count:1 [446]

OlympusSIS2(34853/0x8825) LONG count:1 [46958]


tag count: 2
JPEGFromRawStar